In [1]:
from wmb import cemba
import pandas as pd
from ALLCools.dmr.rms_test import permute_root_mean_square_test_and_estimate_p, downsample_table
import joblib
import pathlib
from tqdm import tqdm
from ALLCools.mcds import RegionDS
import xarray as xr
from collections import defaultdict
import dask
import numpy as np
from tqdm import tqdm

In [2]:
def gen_dmr_bed(dmr_list):
    chrs_list= []
    dmr_start_list= []
    dmr_end_list = []

    for dmr in dmr_list:
        group, start_length = dmr.split(':')
        _chr, group_start = group.split('-')
        dist_to_group_start, length = start_length.split('-')
        dmr_start = int(group_start) + int(dist_to_group_start)
        dmr_end = dmr_start + int(length)
        chrs_list.append(_chr)
        dmr_start_list.append(dmr_start)
        dmr_end_list.append(dmr_end)
    tmp_dmr_df = pd.DataFrame({'chrom':chrs_list,'start':dmr_start_list,'end':dmr_end_list})
    return tmp_dmr_df

## get example celltype bed

In [12]:
#use_dmrs = pd.read_csv('CEMBA.mC.grouped-hypo-DMR.bed', sep = '\t', header = None)

In [3]:
use_dmrs = joblib.load('use_dmrs_list')
celltype_dmr_zarr_path =  '/cemba/wmb/genome/DMR/CEMBA.snmC.AllGroupedDMRs.CellType.mC.zarr/'
# use_celltypes = ["Pvalb Gaba", "MSN D1 Gaba", "STR-PAL Chst9 Gaba","Oligo NN", 
#                  "CA1-ProS Glut", "L6b/CT ENT Glut","Car3 Glut",
#                    "L2/3 IT CTX Glut","L5 NP CTX Glut"]
use_celltypes = ["MSN D1 Sema5a Gaba", "SUB-ProS Glut"]
len(use_dmrs), len(use_celltypes)

(2564356, 2)

In [4]:
mc_ds = xr.open_zarr(celltype_dmr_zarr_path)
_ds = [mc_ds]
dmr_ds = RegionDS(xr.merge(_ds))
dmr_ds.region_dim = 'dmr'
dmr_ds

<xarray.RegionDS>
Dimensions:     (CellType: 261, count_type: 2, dmr: 8075960)
Coordinates:
  * CellType    (CellType) object 'AAA-BST Six3 Meis2 Gaba' ... 'ZI Pax6 Gaba'
  * count_type  (count_type) <U50 'mc' 'cov'
  * dmr         (dmr) <U50 'chr1-3003581:0-2' ... 'chrX-170860316:0-2'
    dmr_chrom   (dmr) <U50 dask.array<chunksize=(8075960,), meta=np.ndarray>
    dmr_end     (dmr) int64 dask.array<chunksize=(8075960,), meta=np.ndarray>
    dmr_start   (dmr) int64 dask.array<chunksize=(8075960,), meta=np.ndarray>
Data variables:
    data        (CellType, dmr, count_type) uint32 dask.array<chunksize=(5, 8075960, 1), meta=np.ndarray>
    frac        (CellType, dmr) float32 dask.array<chunksize=(5, 8075960), meta=np.ndarray>
Attributes:
    region_dim:  dmr

In [5]:
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dmr_ds = dmr_ds.sel(dmr = use_dmrs, CellType = use_celltypes)

In [6]:
dmr_drac_df = dmr_ds['frac'].to_pandas().T

In [7]:
old_to_new = {'MSN D1 Sema5a Gaba':'STR_D1_Sema5a_Gaba',
              'SUB-ProS Glut':'SUB-ProS_Glut'}

In [9]:
ct_dmr_dict = {}
for ct in use_celltypes:
    save_ct = old_to_new[ct]
    ct_value = dmr_drac_df[ct].sort_values()
    
    percentile_25 = np.percentile(ct_value, 25)
    percentile_75 = np.percentile(ct_value, 75)
    average_value = (percentile_25 + percentile_75)/2
    filter_cutoff = average_value - 0.3
    
    if filter_cutoff <= 0.3: 
        celltype_dmr_select= ct_value[ct_value < 0.3].index.tolist()
    else:
        celltype_dmr_select= ct_value[ct_value < filter_cutoff].index.tolist()
    tmp_dmr_df = gen_dmr_bed(celltype_dmr_select)
    ct_dmr_dict[ct] = celltype_dmr_select
    tmp_dmr_df.to_csv(f'/data/CembaData/celltype_dmr_bed/{save_ct}_Hypo.DMR.bed', sep = '\t', header = None, index= None)

In [10]:
import joblib
joblib.dump(ct_dmr_dict, 'ct_dmr_dict')

['ct_dmr_dict']

## get all hypo dmr

In [3]:
celltype_dmr_zarr_path =  '/cemba/wmb/genome/DMR/CEMBA.snmC.AllGroupedDMRs.CellType.mC.zarr/'
use_dmrs = joblib.load('_/use_dmrs_list')

In [4]:
mc_annot = cemba.get_mc_annot()

In [5]:
min_cell_num = 1000

In [6]:
use_celltypes = mc_annot['CellType'].to_pandas().value_counts()[mc_annot['CellType'].to_pandas().value_counts() > min_cell_num].index
len(use_celltypes)

55

In [7]:
# all_celltypes = mc_annot['CellType'].to_pandas().unique().tolist()
# import random
# use_celltypes = random.sample(all_celltypes, 50)

In [8]:
import joblib
joblib.dump(use_celltypes,'use_celltypes_list')

['use_celltypes_list']

In [9]:
mc_ds = xr.open_zarr(celltype_dmr_zarr_path)
_ds = [mc_ds]
dmr_ds = RegionDS(xr.merge(_ds))
dmr_ds.region_dim = 'dmr'
dmr_ds

<xarray.RegionDS>
Dimensions:     (CellType: 261, count_type: 2, dmr: 8075960)
Coordinates:
  * CellType    (CellType) object 'AAA-BST Six3 Meis2 Gaba' ... 'ZI Pax6 Gaba'
  * count_type  (count_type) <U50 'mc' 'cov'
  * dmr         (dmr) <U50 'chr1-3003581:0-2' ... 'chrX-170860316:0-2'
    dmr_chrom   (dmr) <U50 dask.array<chunksize=(8075960,), meta=np.ndarray>
    dmr_end     (dmr) int64 dask.array<chunksize=(8075960,), meta=np.ndarray>
    dmr_start   (dmr) int64 dask.array<chunksize=(8075960,), meta=np.ndarray>
Data variables:
    data        (CellType, dmr, count_type) uint32 dask.array<chunksize=(5, 8075960, 1), meta=np.ndarray>
    frac        (CellType, dmr) float32 dask.array<chunksize=(5, 8075960), meta=np.ndarray>
Attributes:
    region_dim:  dmr

In [10]:
dmr_ds = dmr_ds.sel(dmr = dmr_ds.get_index('dmr').isin(use_dmrs), CellType =use_celltypes)

In [11]:
dmr_drac_df = dmr_ds['frac'].to_pandas().T

In [12]:
dmr_drac_df.head()

CellType,L2/3 IT CTX Glut,L4/5 IT CTX Glut,L6 CT CTX Glut,L2/3 IT ENTl-PIR Glut,Oligo NN,DG Glut,L6 IT CTX Glut,MSN D1 Gaba,TH Prkcd Grin2c Glut,L5 IT CTX Glut,...,SNc-VTA-RAmb Foxa1 Dopa,L5/6 IT TPE-ENT Glut,MOB-STR-CTX Inh IMN,Golgi Gly-Gaba,VLMC NN,L2/3 IT PPP Glut,SI-MPO-LPO Lhx8 Gaba,COAa-PAA Barhl2 Glut,Sncg Gaba,SCs Dmbx1 Gaba
dmr,,,,,,,,,,,,,,,,,,,,,
chr1-3003581:0-2,0.980608,0.972669,0.986928,0.973418,0.921260,0.939698,0.978431,0.991453,0.976242,0.975610,...,0.989011,1.000000,1.000000,0.988095,0.937500,0.973684,1.000000,0.959459,0.986301,0.974026
chr1-3003639:0-2,0.281071,0.337714,0.352789,0.463855,0.117747,0.131926,0.334365,0.423077,0.590580,0.395349,...,0.474576,0.435484,0.181818,0.603773,0.175439,0.428571,0.424242,0.625000,0.428571,0.406780
chr1-3003720:0-179,0.966928,0.971207,0.973765,0.971518,0.847382,0.929297,0.970229,0.963100,0.973684,0.982569,...,0.980663,0.974820,0.920548,0.954338,0.827119,0.971731,0.940171,0.956896,0.979592,0.969799
chr1-3005997:0-2,0.916039,0.868472,0.891873,0.918082,0.513340,0.734488,0.850772,0.812623,0.976015,0.941884,...,0.970874,0.971154,0.483871,0.978022,0.587156,0.971014,0.792453,1.000000,0.929412,0.863636
chr1-3006781:0-2,0.680519,0.728707,0.948276,0.941176,0.872146,0.877551,0.906542,0.854545,0.977444,0.890110,...,0.925926,0.925926,0.916666,0.833333,0.862069,0.882352,0.965517,1.000000,0.999999,0.999999


In [13]:
all_hypo_dmr = []
for ct in tqdm(dmr_drac_df.columns):
    save_ct = ct.replace('/','').replace(' ','-')
    ct_value = dmr_drac_df[ct].sort_values()
    
    percentile_25 = np.percentile(ct_value, 25)
    percentile_75 = np.percentile(ct_value, 75)
    average_value = (percentile_25 + percentile_75)/2
    filter_cutoff = average_value - 0.3
    
    if filter_cutoff <= 0.3: 
        celltype_dmr_select= ct_value[ct_value < 0.3].index.tolist()
    else:
        celltype_dmr_select= ct_value[ct_value < filter_cutoff].index.tolist()
    all_hypo_dmr += celltype_dmr_select   

100%|██████████| 55/55 [00:30<00:00,  1.78it/s]


In [14]:
len(all_hypo_dmr)

14316301

In [15]:
all_hypo_dmr = list(set(all_hypo_dmr))
len(all_hypo_dmr)

1799129

In [16]:
joblib.dump(all_hypo_dmr, 'example_ct_hypo_dmr_list')

['example_ct_hypo_dmr_list']

In [17]:
# all_hypo_dmr_df = gen_dmr_bed(all_hypo_dmr)
# all_hypo_dmr_df.to_csv('CEMBA.mC.grouped-hypo-DMR.bed', sep = '\t', header = None, index= None)
# all_hypo_dmr_df